In [1]:
import os
import re
import json
import torch
import pandas as pd
from PIL import Image
from transformers import AutoProcessor, LlavaOnevisionForConditionalGeneration

# Function to extract a number (optionally followed by meter units) from text.
def extract_meter(text):
    """
    Extracts the first occurrence of a number (optionally followed by 'meters', 'meter', or 'm')
    from the given text and returns it as a float.
    """
    match = re.search(r"(\d+(?:\.\d+)?)\s*(?:meters?|m)?", text, re.IGNORECASE)
    if match:
        try:
            return float(match.group(1))
        except Exception as e:
            print("Error converting extracted value:", e)
    return None

def inference_llava_image(model, processor, image_source, human_text, use_url=False):
    """
    Run inference on a single image using Llava Onevision.
    
    Args:
        model: The Llava Onevision model.
        processor: The processor for formatting the prompt and image.
        image_source (str): If use_url is True, this is the image URL; otherwise, it's the local image file path.
        human_text (str): The text prompt (with any "<image>" marker removed).
        use_url (bool): Whether the image_source is a URL.
    
    Returns:
        output_text (str): The generated answer from the model.
    """
    print(human_text)
    # For a local image, load it and use an image placeholder.
    image = Image.open(image_source).convert("RGB")
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": human_text},
            ],
        },
    ]
    # First, build the text prompt with tokenization.
    inputs = processor.apply_chat_template(
        conversation,
        add_generation_prompt=True,
    )
    # Then, pass the loaded image along with the prompt.
    inputs = processor(
        text=inputs,
        images=image,
        return_tensors="pt"
    ).to(0, torch.float16)
    
    # Generate the model output.
    output_ids = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    
    print("HALLO")
    output_text = processor.decode(output_ids[0], skip_special_tokens=True)
    return output_text

def run_llava_inference_on_dataset(model, processor, dataset_path, image_root, save_name, use_url=False):
    """
    Iterate over items in the test JSON file, run inference, extract meter values,
    compute the absolute error against the ground truth, and save results.
    
    Args:
        model: The Llava Onevision model.
        processor: The processor.
        dataset_path (str): Path to the JSON file (e.g., "./dataset_new/test.json").
        image_root (str): Root directory for images (if not using URL).
        save_name (str): Base name for the output CSV file.
        use_url (bool): Whether the image paths in the dataset are URLs.
    
    Returns:
        df: A pandas DataFrame with columns: id, predicted_meter, ground_truth_meter, and error.
    """
    results = []
    
    with open(dataset_path, "r") as f:
        data = json.load(f)
    
    for item in data:
        item_id = item.get("id")
        image_file = item["image"]
        image_source = image_file if use_url else os.path.join(image_root, image_file.lstrip("/"))
        
        # Extract the human prompt.
        human_text = ""
        for conv in item.get("conversations", []):
            if conv.get("from", "").lower() == "human":
                human_text = conv.get("value", "")
                break
        if human_text.startswith("<image>"):
            human_text = human_text[len("<image>"):].strip()
        
        # Extract the ground truth answer from the first GPT message.
        ground_truth_str = ""
        for conv in item.get("conversations", []):
            if conv.get("from", "").lower() == "gpt":
                ground_truth_str = conv.get("value", "")
                break
        
        
        model_output = inference_llava_image(model, processor, image_source, human_text, use_url=use_url)

        
        pred_meter = extract_meter(model_output)
        gt_meter = extract_meter(ground_truth_str)
        error = abs(pred_meter - gt_meter) if (pred_meter is not None and gt_meter is not None) else None
        
        print(f"Item ID: {item_id}")
        print("Predicted Meter:", pred_meter)
        print("Ground Truth:", gt_meter)
        print("Error:", error)
        print("=" * 50)
        
        results.append({
            "id": item_id,
            "predicted_meter": pred_meter,
            "ground_truth_meter": gt_meter,
            "error": error
        })
    
    # Save the results to a CSV file.
    df = pd.DataFrame(results)
    csv_filename = f"res/{save_name}.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Results saved to {csv_filename}")
    
    return df



/hpi/fs00/home/liudvikas.zekas/miniconda3/envs/lmms-finetune/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_json_path = "./dataset_new/test.json"
image_root = "/"
processor = AutoProcessor.from_pretrained("llava-hf/llava-onevision-qwen2-7b-ov-hf")

In [3]:
base_model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    "llava-hf/llava-onevision-qwen2-7b-ov-hf",
    torch_dtype="auto",
    device_map="auto",
    cache_dir="/hpi/fs00/scratch/liudvikas.zekas/.cache"
)

print("=== Inference using the Base onevision Model ===")
df_base = run_llava_inference_on_dataset(base_model, processor, test_json_path, image_root, "llava_onevision_7b_no_inference")

# Optionally, delete the model to free GPU memory.
del base_model

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:29<00:00,  7.42s/it]


=== Inference using the Base onevision Model ===
How far is the gas station away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


HALLO
Item ID: ugebwzaohO3D7XbW343_qA_back
Predicted Meter: None
Ground Truth: 30.0
Error: None
How far is the post box away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

HALLO
Item ID: C72ymm34PyRc2OruySafpQ_left
Predicted Meter: 10.0
Ground Truth: 15.0
Error: 5.0
How far is the gas station away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


HALLO
Item ID: tL1cMj27xoKEUPU8emSdSQ_right
Predicted Meter: 100.0
Ground Truth: 22.0
Error: 78.0
How far is the gas station away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


HALLO
Item ID: 1KKSCQJLKYDDK3KSdFkHdw_right
Predicted Meter: 150.0
Ground Truth: 23.0
Error: 127.0
How far is the gas station away from the camera in meters, rounded to the next meter?
HALLO
Item ID: l7wQW6Nrj2lN4nrgHrVA0g_left
Predicted Meter: 100.0
Ground Truth: 26.0
Error: 74.0
Results saved to res/llava_onevision_7b_no_inference.csv


In [7]:
del finetuned_model

In [9]:
finetuned_model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    "/hpi/fs00/scratch/liudvikas.zekas/checkpoints/llava-onevision-7b-ov_lora-True_qlora-False-custom", 
    torch_dtype="auto",
    low_cpu_mem_usage=True, 
    cache_dir="/hpi/fs00/scratch/liudvikas.zekas/.cache"
).to(0)

print("=== Inference using the Finetuned Qwen Model ===")
df_finetuned = run_llava_inference_on_dataset(finetuned_model, processor, test_json_path, image_root, "llava_onevision_7b_inference")

# Clean up.
del finetuned_model

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.06it/s]


=== Inference using the Finetuned Qwen Model ===
How far is the gas station away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


HALLO
Item ID: ugebwzaohO3D7XbW343_qA_back
Predicted Meter: 29.0
Ground Truth: 30.0
Error: 1.0
How far is the post box away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

HALLO
Item ID: C72ymm34PyRc2OruySafpQ_left
Predicted Meter: 14.0
Ground Truth: 15.0
Error: 1.0
How far is the gas station away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


HALLO
Item ID: tL1cMj27xoKEUPU8emSdSQ_right
Predicted Meter: 20.0
Ground Truth: 22.0
Error: 2.0
How far is the gas station away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


HALLO
Item ID: 1KKSCQJLKYDDK3KSdFkHdw_right
Predicted Meter: 27.0
Ground Truth: 23.0
Error: 4.0
How far is the gas station away from the camera in meters, rounded to the next meter?
HALLO
Item ID: l7wQW6Nrj2lN4nrgHrVA0g_left
Predicted Meter: 29.0
Ground Truth: 26.0
Error: 3.0
Results saved to res/llava_onevision_7b_inference.csv


In [10]:
base_model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    "llava-hf/llava-onevision-qwen2-0.5b-ov-hf",
    torch_dtype="auto",
    device_map="auto",
    cache_dir="/hpi/fs00/scratch/liudvikas.zekas/.cache"
)

print("=== Inference using the Base onevision Model ===")
df_base = run_llava_inference_on_dataset(base_model, processor, test_json_path, image_root, "llava_onevision_0.5b_no_inference")

# Optionally, delete the model to free GPU memory.
del base_model

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


=== Inference using the Base onevision Model ===
How far is the gas station away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


HALLO
Item ID: ugebwzaohO3D7XbW343_qA_back
Predicted Meter: 1.0
Ground Truth: 30.0
Error: 29.0
How far is the post box away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

HALLO
Item ID: C72ymm34PyRc2OruySafpQ_left
Predicted Meter: 1.0
Ground Truth: 15.0
Error: 14.0
How far is the gas station away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


HALLO
Item ID: tL1cMj27xoKEUPU8emSdSQ_right
Predicted Meter: 1.0
Ground Truth: 22.0
Error: 21.0
How far is the gas station away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


HALLO
Item ID: 1KKSCQJLKYDDK3KSdFkHdw_right
Predicted Meter: 1.0
Ground Truth: 23.0
Error: 22.0
How far is the gas station away from the camera in meters, rounded to the next meter?
HALLO
Item ID: l7wQW6Nrj2lN4nrgHrVA0g_left
Predicted Meter: 1.0
Ground Truth: 26.0
Error: 25.0
Results saved to res/llava_onevision_0.5b_no_inference.csv


In [12]:
finetuned_model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    "/hpi/fs00/scratch/liudvikas.zekas/checkpoints/llava-onevision-0.5b-ov_lora-True_qlora-False-custom", 
    torch_dtype="auto",
    low_cpu_mem_usage=True, 
    cache_dir="/hpi/fs00/scratch/liudvikas.zekas/.cache"
).to(0)

print("=== Inference using the Finetuned Qwen Model ===")
df_finetuned = run_llava_inference_on_dataset(finetuned_model, processor, test_json_path, image_root, "llava_onevision_0.5b_inference")

# Clean up.
del finetuned_model

=== Inference using the Finetuned Qwen Model ===
How far is the gas station away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


HALLO
Item ID: ugebwzaohO3D7XbW343_qA_back
Predicted Meter: 29.0
Ground Truth: 30.0
Error: 1.0
How far is the post box away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


HALLO
Item ID: qz33ZhqXW5DY-2hPzv2CMQ_back
Predicted Meter: 12.0
Ground Truth: 10.0
Error: 2.0
How far is the gas station away from the camera in meters, rounded to the next meter?


...

HALLO
Item ID: C72ymm34PyRc2OruySafpQ_left
Predicted Meter: 27.0
Ground Truth: 15.0
Error: 12.0
How far is the gas station away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


HALLO
Item ID: tL1cMj27xoKEUPU8emSdSQ_right
Predicted Meter: 19.0
Ground Truth: 22.0
Error: 3.0
How far is the gas station away from the camera in meters, rounded to the next meter?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


HALLO
Item ID: 1KKSCQJLKYDDK3KSdFkHdw_right
Predicted Meter: 26.0
Ground Truth: 23.0
Error: 3.0
How far is the gas station away from the camera in meters, rounded to the next meter?
HALLO
Item ID: l7wQW6Nrj2lN4nrgHrVA0g_left
Predicted Meter: 29.0
Ground Truth: 26.0
Error: 3.0
Results saved to res/llava_onevision_0.5b_inference.csv
